# Assignment 4: CNN

## Description

Implement a Convolutional Neural Network (CNN) classifier to predict whether a given icon image is the real / fake. Where the fake images were generated by TAs with a neural network.

- You are not required to use Colab in this assignment, but you have to **submit your source code**.

## Dataset

- https://lab.djosix.com/icons.zip
- 64x64 RGB jpg images


```
real/           (10000 images)
    0000.jpg
    0001.jpg
    ...
    9999.jpg
fake/           (10000 images)
    0000.jpg
    0001.jpg
    ...
    9999.jpg
unknown/        (5350 images, testing set)
    0000.jpg
    0001.jpg
    ...
    5349.jpg
```

- Training set
  - 20000 icons in `real/` and `fake/`
  - You should predict 1 for icons in `real/` and 0 for icons in `fake/`
- Testing set:
  - 5350 icons in `unknown/`
  - Your score depends on the **accuracy** on this testing set,  
    so the prediction of each icon in `unknown/` should be submitted (totally 5350 predictions, see below).


## Submission

Please upload **2 files** to E3. (`XXXXXXX` is your student ID)

1. **`XXXXXXX_4_result.json`**  
  This file contains your model prediction for the testing set.  
  You must generate this file with the function called `save_predictions()`.
2. **`XXXXXXX_4_source.zip`**  
  Zip your source code into this archive.


## Hints

- **Deep Learning Libraries**: You can use any deep learning frameworks (PyTorch, TensorFlow, ...).
- **How to implement**: There are many CNN examples for beginners on the internet, e.g. official websites of the above libraries, play with them and their model architectures to abtain high accuracy on testing set.
- **GPU/TPU**: Colab provides free TPU/GPU for training speedup, please refer to [this page in `pytut.pdf` on E3](https://i.imgur.com/VsrUh7I.png).


### Include this in your code to generate result file

In [ ]:
import json

def save_predictions(student_id, predictions):
  # Please use this function to generate 'XXXXXXX_4_result.json'
  # `predictions` is a list of int (0 or 1; fake=0 and real=1)
  # For example, `predictions[0]` is the prediction given "unknown/0000.jpg".
  # it will be 1 if your model think it is real, else 0 (fake).

  assert isinstance(student_id, str)
  assert isinstance(predictions, list)
  assert len(predictions) == 5350

  for y in predictions:
    assert y in (0, 1)

  with open('{}_4_result.json'.format(student_id), 'w') as f:
    json.dump(predictions, f)




# code

parameters

In [ ]:
#load dataset
TRAIN_SET_RATIO = 0.7
#model
CLASS_NUM = 13
#training
LEARNING_RATE = 0.0001
MOMENTUM = 0.75
#main
EPOCH_NUM = 30
WORKER_NUM = 2

load dataset

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

# Requirements of pretrained VGG
def my_loader(path):
  img = Image.open(path)
  preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
  ])
  return preprocess(img)

class MyDataset(Dataset):
  def __init__(self, img_paths, labels=[], loader=my_loader):
    self.img_paths = img_paths
    self.labels = labels
    self.loader = loader
  def __getitem__(self, idx):
    path = self.img_paths[idx]
    img = self.loader(path)
    if len(self.labels) == 0:
      return img
    label = self.labels[idx]
    return img, label
  def __len__(self):
    return len(self.img_paths)

def load_dataset():
  if not os.path.exists('icons.zip'):
    url = 'https://lab.djosix.com/icons.zip'
    assert os.system(f'wget {url}') == 0
    assert os.system('unzip icons.zip') == 0

  # Both ./fake and ./real contain 10000 icons
  total = 10000
  train_set_size = int(total * TRAIN_SET_RATIO)
  # ./unknown contains 5350 icons
  test_set_size = 5350

  train_img_paths = []
  val_img_paths = []
  train_labels = []
  val_labels = []
  for label, dir in enumerate(['./fake', './real']):
    train_img_paths += [f'{dir}/{n:04}.jpg' for n in range(train_set_size)]
    val_img_paths += [f'{dir}/{n:04}.jpg' for n in range(train_set_size, total)]
    train_labels += [label] * train_set_size
    val_labels += [label] * (total - train_set_size)

  test_img_paths = [f'./unknown/{n:04}.jpg' for n in range(test_set_size)]

  train_set = MyDataset(train_img_paths, train_labels)
  val_set = MyDataset(val_img_paths, val_labels)
  test_set = MyDataset(test_img_paths)

  return train_set, val_set, test_set

model

In [ ]:
import torch.nn as nn
from torchvision import models

def set_param_requires_grad(model):
  for param in model.parameters():
    param.requires_grad = False

model = models.densenet161(pretrained=True)
set_param_requires_grad(model)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, CLASS_NUM)
input_size = 224

# Froze feature extracting layers
for param in model.features.parameters():
  param.requires_grad = False

diagram

In [ ]:
import matplotlib.pyplot as plt

def draw_diagram(loss_log, acc_log, epoch_num):
  graph, (plot1, plot2) = plt.subplots(nrows=2)
  plot1.set_title('Loss')
  plot1.plot(loss_log['train'], label='training')
  plot1.plot(loss_log['val'], label='validation')
  plot1.legend()
  plot2.set_title('Accuracy')
  plot2.plot(acc_log['train'], label='training')
  plot2.plot(acc_log['val'], label='validation')
  plot2.legend()
  graph.tight_layout()
  plt.show()

training

In [ ]:
"""
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for inputs, lables in train_loader:
        inputs, lables = inputs.to(device), lables.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, lables)
        preds = outputs.max(1).indices
        loss.backward()
        optimizer.step()
def val(model, device, val_loader):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, lables in val_loader:
            inputs, lables = inputs.to(device), lables.to(device)
            output = model(inputs)
            val_loss += F.nll_loss(output, lables, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(lables.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)

    print('\nval set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
"""
import copy

def train_and_validate_model(model, dataloaders, criterion, optimizer, epoch_num=25):
  best_weights = None
  best_acc = 0.0
  loss_log = { 'train': [], 'val': [] }
  acc_log = { 'train': [], 'val': [] }
  
  for epoch in range(epoch_num):
    print(f'Epoch: {epoch + 1}/{epoch_num}')

    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()

      running_loss = 0.0
      running_corrects = 0

      # For each batch of data
      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()

        # Use with statement to temporarily enable gradient calculation
        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          preds = outputs.max(1).indices

          if phase == 'train':
            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

      epoch_loss = running_loss / len(dataloaders[phase].dataset)
      epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

      if phase == 'val' and epoch_acc > best_acc:
        best_weights = copy.deepcopy(model.state_dict())
        best_acc = epoch_acc

      loss_log[phase].append(epoch_loss)
      acc_log[phase].append(epoch_acc)

      print(f'[{phase}] Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

  # Load best model weights
  model.load_state_dict(best_weights)
  return model

main function

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()

train_set, val_set, test_set = load_dataset()
dataloaders = {
    'train': DataLoader(train_set, batch_size=8, shuffle=True, num_workers=WORKER_NUM),
    'val': DataLoader(val_set, batch_size=8, shuffle=True, num_workers=WORKER_NUM),
    'test': DataLoader(test_set, batch_size=8, shuffle=False, num_workers=WORKER_NUM)
}
params_to_update = []
for name, param in model.named_parameters():
  if param.requires_grad:
    params_to_update.append(param)
optimizer = optim.SGD(params_to_update, lr=LEARNING_RATE, momentum=MOMENTUM)

model = train_and_validate_model(model, dataloaders, criterion, optimizer, EPOCH_NUM)


predictions = []
for inputs in dataloaders['test']:
  inputs = inputs.to(device)
  outputs = model(inputs)
  preds = outputs.max(1).indices.tolist()
  predictions += preds

save_predictions('0816004', predictions)